In [ ]:
"""
Focusing on things that were not done in v1.

"""


In [5]:
%load_ext autoreload
%autoreload 2

from neuralmonkey.classes.session import load_mult_session_helper
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

#%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# load all sessions for Pancho on 221020
MS = load_mult_session_helper(221020, 'Pancho')

Searching using this string:
/mnt/Freiwald/ltian/recordings/*Pancho*/*221020*/**
Found this many paths:
2
---
/mnt/Freiwald/ltian/recordings/Pancho/221020/Pancho-221020-133436
---
/mnt/Freiwald/ltian/recordings/Pancho/221020/Pancho-221020-161238
session:  0
------------------------------
Loading this neural session: 0
Loading these beh expts: None
Loading these beh sessions: None
Using this beh_trial_map_list: None
Searching using this string:
/gorilla1/neural_preprocess/recordings/*Pancho*/*221020*/**
Found this many paths:
2
---
/gorilla1/neural_preprocess/recordings/Pancho/221020/Pancho-221020-133436
---
/gorilla1/neural_preprocess/recordings/Pancho/221020/Pancho-221020-161238
{'filename_components_hyphened': ['Pancho', '221020', '133436'], 'basedirs': ['/gorilla1/neural_preprocess/recordings/Pancho', '/gorilla1/neural_preprocess/recordings/Pancho/221020'], 'basedirs_filenames': ['221020', 'Pancho-221020-133436'], 'filename_final_ext': 'Pancho-221020-133436', 'filename_final_noext':

** Loading tank data from local (previusly cached)
== Done
== Trying to load events data
Loading this events (pd) locally to:  /gorilla1/neural_preprocess/recordings/Pancho/221020/Pancho-221020-161238/events_photodiode.pkl
== Done
** MINIMAL_LOADING, therefore loading previuosly cached data
Generated self._MapperTrialcode2TrialToTrial!
Extracted into self.Dat[epoch_orig]
=== CLEANING UP self.Dat (_cleanup_reloading_saved_state) ===== 
0 _behclass_alignsim_compute
Running D._behclass_tokens_extract_datsegs
0 _behclass_tokens_extract_datsegs
stored in self.Dat[BehClass]
Extracted successfully for session:  1
Generated index mappers!


### Saving local cached data

In [ ]:

sn = MS.SessionsList[0]
sn._MapperTrialcode2TrialToTrial

In [11]:
for sn in MS.SessionsList:
    sn._savelocalcached_save(save_dataset_beh=False, save_datslices=False, ONLY_EXTRA_STUFF=True)

### Snippets

Collects data aligned to instances of a given event. Can then use the Snippets methods to do many things, plotting, analysis, etc.

In [ ]:
from neuralmonkey.classes.snippets import Snippets, extraction_helper


In [ ]:
# Pick out a single session
sn = MS.SessionsList[0]


In [ ]:
# Turn on debug mode, this prunes sites and trials, so below runs faster
DEBUG = True
sn._DEBUG_PRUNE_SITES = DEBUG
sn._DEBUG_PRUNE_TRIALS = DEBUG


In [ ]:
# ****** FOR KEDAR: you would want to modify list_events to something like this:
list_events = ["saccon", "saccoff"]


In [ ]:
# Main code for extracting Snippets (SP)

# == MODIFY THESE:
list_events = ["go"]
list_pre_dur = [-0.5]
list_post_dur = [0.5]

# == IGNORE THESE:
which_level = "flex"
list_features_extraction = []
list_features_get_conjunction = []
trials_prune_just_those_including_events=False

SP = Snippets(sn,
    which_level,
    list_events,
    list_features_extraction,
    list_features_get_conjunction,
    list_pre_dur,
    list_post_dur,
    strokes_only_keep_single=False,
    tasks_only_keep_these=None,
    prune_feature_levels_min_n_trials=None,
    dataset_pruned_for_trial_analysis=None,
    trials_prune_just_those_including_events=trials_prune_just_those_including_events,
    fr_which_version='sqrt',
    NEW_VERSION=True,
    SKIP_DATA_EXTRACTION=False
             )


In [ ]:
# Inspect the data. 
# Each row represents a single combination of:
# (trial, chan, event). To see that, inspect the output of 

display(SP.DfScalar)

from pythonlib.tools.pandastools import grouping_print_n_samples
grouping_print_n_samples(SP.DfScalar, ["trialcode", "chan", "event_aligned"])


##### Plotting functions for Snippets

In [ ]:
SP.plotgood_rasters(chan)

In [ ]:
site =2
event=None


fig, axesall = plt.subplots(1,2)

# 1) Plot the rasters ont he top row.
# axes = axesall[0]
ax = axesall.flatten()[0]
SP.plotgood_rasters(site, event, ax)    

# SP.plotgood_rasters(chan)

### Computing modulation of activity by certain variables

For each site, compute how strongly it is modulated by a specific variable, computed 
separatedly for each event.

In [ ]:
# First, append the variables that you wish to analyze
SP.datasetbeh_append_column("epoch", sn.Datasetbeh) # assign epoch to SP


In [ ]:
if DEBUG:
    # if you debug, then you have small n trials, so lower the threshold for keeping data.
    print(SP.ParamsGlobals)
    SP.globals_update(globals_nmin=2)


In [ ]:
# Run this to compute modulation
DF_VAR, DF_FR, DF_FR_LEVELS, list_eventwindow_event = SP.modulationgood_compute_wrapper("epoch", 
                                                                                        DEBUG_CONJUNCTIONS=False, 
                                                                                        THINGS_TO_EXTRACT=["anova"])


In [ ]:
# Plot results and save
sdir_base = "/tmp"
SP.modulationgood_plot_WRAPPER(DF_VAR, DF_FR, DF_FR_LEVELS, list_eventwindow_event, 
                               var="epoch", vars_conjuction=["dummy_var"],
                               sdir_base=sdir_base, N_WAYS=1)

### Modulation by time (relative to aligned events)

For each site x event, compute a scalar value indiicating how much its actiivyt is modulated
with respect to this event. This moudlation can be increase in fr, decrease, or any
activity pattern that is consistent across trials.

In [ ]:
# Extract modulation, for each site x event

In [ ]:
from neuralmonkey.analyses.event_temporal_modulation_v2 import preprocess_and_extract
from neuralmonkey.analyses.event_temporal_modulation import plot_overview

df_modtime = preprocess_and_extract(SP)


In [ ]:
# Plot overview of modulation

In [ ]:
from neuralmonkey.analyses.event_temporal_modulation_v2 import plot_overview
SAVEDIR = "/tmp"
##################### PLOTS
plot_overview(df_modtime, SP, SAVEDIR)